<a href="https://colab.research.google.com/github/xingji1337/week6HO/blob/main/Week6_3_Streamlit_App2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 6 — Streamlit App (Graph-RAG + Multi-Hop)
**Goal:** Expose your advanced pipeline via a simple web app.


In [ ]:
!pkill -f streamlit || true
!pkill -f cloudflared || true

^C
^C


In [ ]:
!pip -q install streamlit==1.38.0 sentence-transformers networkx spacy
!python -m spacy download en_core_web_sm
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 139.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
# 0) (optional) see what's installed
!pip show streamlit watchdog google-adk

# 1) remove the conflicting package
!pip uninstall -y google-adk

# 2) make sure Streamlit gets a compatible watchdog
!pip install -U "watchdog<5,>=2.1.5" "streamlit==1.38.0"

# 3) confirm versions
import streamlit, watchdog

Name: streamlit
Version: 1.38.0
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, rich, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 
---
Name: watchdog
Version: 4.0.2
Summary: Filesystem events monitoring
Home-page: https://github.com/gorakhargosh/watchdog
Author: Yesudeep Mangalapilly
Author-email: yesudeep@gmail.com
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: 
Required-by: streamlit


In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# TODO: change this to where you put the files
%cd /content/drive/MyDrive
!ls -la

Mounted at /content/drive
/content/drive/MyDrive
total 747745
-rw------- 1 root root       176 Jan  5  2010 '10 Things Not to Buy in 2010.gdoc'
-rw------- 1 root root       176 Mar 13  2008 '2005 CERNER SCHOLARSHIP PROGRAM.gdoc'
-rw------- 1 root root    403456 Nov 26  2012 'academic path_final.doc'
-rw------- 1 root root      7740 Oct  7 00:10  app_track_c_min.py
-rw------- 1 root root       176 Oct  4  2007 'A Whole New World.gdoc'
-rw------- 1 root root       176 Dec 10  2009 'Chapter 12 Personality.gdoc'
-rwx------ 1 root root  41192406 Sep 22 13:36  cloudflared-linux-amd64
-rw------- 1 root root  41192406 Sep 22 13:36  cloudflared-linux-amd64.1
-rw------- 1 root root  41192406 Sep 22 13:36  cloudflared-linux-amd64.2
drwx------ 2 root root      4096 Jan 31  2024 'Colab Notebooks'
-rw------- 1 root root       176 Nov 13  2024 'Copy of  Combat Achievements Per Region (1).gsheet'
-rw------- 1 root root       176 Nov 22  2024 'Copy of  Combat Achievements Per Region.gsheet'
-rw------- 

In [7]:
# Start Streamlit in the background on port 8501
!streamlit run app_track_c_min.py --server.port 8501 --server.headless true &>/content/streamlit.log &

# Give it a moment, then show the first part of the log
!sleep 3
!sed -n '1,200p' /content/streamlit.log




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.255.237:8501



In [8]:
!curl -I http://localhost:8501


HTTP/1.1 200 OK
Server: TornadoServer/6.4.2
Content-Type: text/html
Date: Tue, 07 Oct 2025 00:53:13 GMT
Accept-Ranges: bytes
Etag: "dc6df26b54ef96810fd4ff9271556d7ea55b11d9cc0f482bf926e6799f6e76a78d91a2c3cc93aa97e9be16c46ab15f66ac11789a762bde442c063c90712e860d"
Last-Modified: Tue, 07 Oct 2025 00:28:02 GMT
Cache-Control: no-cache
Content-Length: 891
Vary: Accept-Encoding



In [9]:
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501 --no-autoupdate

2025-10-07T00:53:15Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-07T00:53:15Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-07T00:53:18Z INF +--------------------------------------------------------------------------------------------+
2025-10-07T00:53:18Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-07T00:53:18Z INF |  https://upon-participation-frankfurt-vacations.tryclo

In [ ]:
!sed -n '1,120p' /content/streamlit.log